<p align="left">
<img width="50%" src="https://drive.google.com/uc?export=view&id=10y3NKbbk7yt7cZDMszMt04g6NquTEa4p" alt="Carbon Logo" />
</p>


# Carbon Simulator - Litepaper Examples

Those are the examples from the [Litepaper][lp] "Use Cases" section, presented very concisely. Please see the book `CarbonSim-Examples.ipynb` for a longer discussion, more technical explanations, and additional examples.

[lp]:https://carbondefi.xyz/r/litepaper

In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v1.3 (30/Nov/2022)


## LP Example: Limit Order

**Bob creates a strategy where he buys ETH when the price goes to 1500 USDC. He is willing to spend 5000 USDC for this. He would like to buy at this specific price only.**

In [2]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [3]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=10000000, psell_end=10000000, amt_buy=10000, pbuy_start=1500, pbuy_end=1500)["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.0,0.0,ETH,10000000.0,10000000.0,10000000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,10000.0,10000.0,USDC,1500.0,1500.0,1500.0,USDC per ETH,0


The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [4]:
for size in [2,1.4,3,0.5]:
    result = Sim.trader_sells("ETH", size)
result

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

In [5]:
Sim.state()["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,0,0,A,AMM sells 3000USDC buys 2ETH,True,True,None,3000.0,USDC,2.0,ETH,ETHUSDC,[1],1,1500.0,USDC per ETH,None
0,1,1,A,AMM sells 2100USDC buys 1ETH,True,True,None,2100.0,USDC,1.4,ETH,ETHUSDC,[1],1,1500.0,USDC per ETH,None
0,2,2,A,AMM sells 4500USDC buys 3ETH,True,True,None,4500.0,USDC,3.0,ETH,ETHUSDC,[1],1,1500.0,USDC per ETH,None


After those trades, Bob purchased 6.4 ETH. This should have cost him USDC 9600. It did.

In [6]:
print(6.4*1500)
Sim.state()["orders"].query("y_unit=='ETH'")["y"]

9600.0


0    6.4
Name: y, dtype: float64

We confirm that: Bob as 400 USDC left from the initial 10000

In [7]:
print(10000-6.4*1500)
Sim.state()["orders"].query("y_unit=='USDC'")["y"]

400.0


1    400.0
Name: y, dtype: float64

## LP Example: "Buy-Low Sell-High" Strategy

**Alice creates a strategy where she buys ETH as the price goes between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this. She also wants to sell her acquired ETH when the price goes from 2000 to 2500 USDC.**

In [8]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [9]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=2000, psell_end=2500, amt_buy=5000, pbuy_start=1600, pbuy_end=1500)["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.0,0.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.0,5000.0,USDC,1600.0,1500.0,1600.0,USDC per ETH,0


The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [10]:
for size in [1,1,1,1]:
    result = Sim.trader_sells(tkn="ETH", amt=size, pair='ETHUSDC')
result

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

In [11]:
Sim.state()["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,0,0,A,AMM sells 1584USDC buys 1ETH,True,True,None,1583.905411,USDC,1.0,ETH,ETHUSDC,[1],1,1583.905411,USDC per ETH,None
0,1,1,A,AMM sells 1552USDC buys 1ETH,True,True,None,1552.357373,USDC,1.0,ETH,ETHUSDC,[1],1,1552.357373,USDC per ETH,None
0,2,2,A,AMM sells 1522USDC buys 1ETH,True,True,None,1521.742595,USDC,1.0,ETH,ETHUSDC,[1],1,1521.742595,USDC per ETH,None


Looking at the order(s) we see that the "sell USDC, buy ETH" range is mostly empty -- she has 341.99 USDC left from here initial 5000. We also see that the marginal price of the range is now at 1506.7 which is very close to the end point of the range at 1500, so the last dollar she will sell expensively (buy ETH cheaply). If she can of course.

In [12]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,3.0,3.000000,ETH,2000.0,2500.0,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.0,341.994621,USDC,1600.0,1500.0,1506.737091,USDC per ETH,0


Specifically, after those trades, Alice has 3 ETH, and she has the aforemention 341.99 USDC left.

In [13]:
Sim.state()["orders"][['y', 'y_unit']]

,y,y_unit
0,3.000000,ETH
1,341.994621,USDC


Then the market turns, and it buys ETH at the price the Alice is willing to sell it. We see that the current marginal price of her "sell ETH position" is at 2000. In other words -- the first infinitesimal unit of ETH will be sold at 2000.

In [14]:
Sim.state()["orders"].query("tkn=='ETH'")

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,3.0,3.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,1


Again the market ("traders") is trading against Alice here, buying ETH. The last trade fails because there is not enough ETH left to sell in Alice's position.

In [15]:
for size in [1,1,1,1]:
    result = Sim.trader_buys(tkn="ETH", amt=size, pair='ETHUSDC')
result

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

In [16]:
Sim.state()["trades"].query("aggr==True").query("tkn1=='ETH'")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,3,3,A,AMM sells 1ETH buys 2073USDC,True,True,None,1.0,ETH,2072.949017,USDC,ETHUSDC,[0],1,2072.949017,USDC per ETH,None
0,4,4,A,AMM sells 1ETH buys 2230USDC,True,True,None,1.0,ETH,2229.893067,USDC,ETHUSDC,[0],1,2229.893067,USDC per ETH,None
0,5,5,A,AMM sells 1ETH buys 2405USDC,True,True,None,1.0,ETH,2405.361849,USDC,ETHUSDC,[0],1,2405.361849,USDC per ETH,None


At the end, Alice has 7050 USDC, against a starting value of 5000.

In [17]:
Sim.state()["orders"][['y', 'y_unit']]

,y,y_unit
0,0.000000,ETH
1,7050.198554,USDC


## LP Example: "Average-In" Order

**Jane creates a strategy to buy ETH between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this.**

Note: this is simply Alice's strategy without the sell part.

In [18]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [19]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=1000000, psell_end=1000000, amt_buy=5000, pbuy_start=1600, pbuy_end=1500)["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.0,0.0,ETH,1000000.0,1000000.0,1000000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.0,5000.0,USDC,1600.0,1500.0,1600.0,USDC per ETH,0


The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [20]:
for size in [1,1,1,340/1500]:
    result = Sim.trader_sells(tkn="ETH", amt=size, pair="ETHUSDC")
result

{'success': True,
 'trades':    uid  id subid                         note   aggr  exec limitfail  \
 0  3.0   3     0                     route #1  False  True      None   
 0    3   3     A  AMM sells 341USDC buys 0ETH   True  True      None   
 
          amt1  tkn1      amt2 tkn2     pair routeix  nroutes        price  \
 0  340.765429  USDC  0.226667  ETH  ETHUSDC       1        1  1503.376892   
 0  340.765429  USDC  0.226667  ETH  ETHUSDC     [1]        1  1503.376892   
 
          p_unit threshold_orders  
 0  USDC per ETH             None  
 0  USDC per ETH             None  }

In [21]:
Sim.state()["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,0,0,A,AMM sells 1584USDC buys 1ETH,True,True,None,1583.905411,USDC,1.000000,ETH,ETHUSDC,[1],1,1583.905411,USDC per ETH,None
0,1,1,A,AMM sells 1552USDC buys 1ETH,True,True,None,1552.357373,USDC,1.000000,ETH,ETHUSDC,[1],1,1552.357373,USDC per ETH,None
0,2,2,A,AMM sells 1522USDC buys 1ETH,True,True,None,1521.742595,USDC,1.000000,ETH,ETHUSDC,[1],1,1521.742595,USDC per ETH,None
0,3,3,A,AMM sells 341USDC buys 0ETH,True,True,None,340.765429,USDC,0.226667,ETH,ETHUSDC,[1],1,1503.376892,USDC per ETH,None


After those trades, Jane has purchased 3.2 ETH, and she has 1.2 USDC left. She bought at an average price of 1549.

In [22]:
o = Sim.state()["orders"][['y', 'y_unit']]
print((5000 - o.iloc[0]['y'])/ o.iloc[0]['y'])
o

1548.5866167782422


,y,y_unit
0,3.226667,ETH
1,1.229192,USDC


## LP Example: Token Distribution Order

**Token project XYZ creates a strategy to sell 1 million units of its XYZ token in the price range of 0.50-2.00. The project would like to “back-load” the distribution process, such that half of the tokens are sold between 1.50-2.00. In total, the project wishes to receive a minimum of 1.37m in cash for its tokens.**

In [23]:
Sim = CarbonSimulatorUI(pair="XYZUSDC")

In [24]:
Sim.add_strategy(tkn="XYZ", amt_sell=500000, psell_start=0.5, psell_end=2, amt_buy=0, pbuy_start=0.001, pbuy_end=0.001, pair='XYZUSDC')
Sim.add_strategy(tkn="XYZ", amt_sell=500000, psell_start=1.5, psell_end=2, amt_buy=0, pbuy_start=0.001, pbuy_end=0.001, pair='XYZUSDC')
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,XYZUSDC,XYZ,500000.0,500000.0,XYZ,0.500,2.000,0.500,USDC per XYZ,1
1,1,XYZUSDC,USDC,0.0,0.0,USDC,0.001,0.001,0.001,USDC per XYZ,0
2,2,XYZUSDC,XYZ,500000.0,500000.0,XYZ,1.500,2.000,1.500,USDC per XYZ,3
3,3,XYZUSDC,USDC,0.0,0.0,USDC,0.001,0.001,0.001,USDC per XYZ,2


The market is now buying

In [25]:
for size in [100]*11:
    result = Sim.trader_buys(tkn="XYZ", amt=size*999.999, pair='XYZUSDC')
result

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

We see that initially the price is rising fast, but as soon as both positions yield tokens (above 1.50) there is more capacity in any given price range

In [26]:
Sim.state()["trades"].query("aggr==True")[["amt1", "tkn1", "nroutes", "price"]]

,amt1,tkn1,nroutes,price
0,99999.9,XYZ,1,0.555555
0,99999.9,XYZ,1,0.694444
0,99999.9,XYZ,1,0.892857
0,99999.9,XYZ,1,1.190475
0,99999.9,XYZ,2,1.50838
0,99999.9,XYZ,2,1.592973
0,99999.9,XYZ,2,1.671989
0,99999.9,XYZ,2,1.757033
0,99999.9,XYZ,2,1.848734
0,99999.9,XYZ,2,1.947808


After those trades, the token project has sold most of its tokens (just 1 left across both positions), and has received USD 1,366,023

In [27]:
o = Sim.state()["orders"][['y', 'y_unit']]
o.query("y_unit=='XYZ'").sum()

y            1.0
y_unit    XYZXYZ
dtype: object

In [28]:
o.query("y_unit=='USDC'").sum()

y         1366023.403785
y_unit          USDCUSDC
dtype: object